In [0]:
# ============================================
# 🥉 BRONZE LAYER - INGESTA DE DATOS CRUDOS
# ============================================
# Proyecto: Startup Death Oracle
# Autor: Sergio Rincón
# Fecha: 2025-12-02
# ============================================

# Importar librerías
import pandas as pd
from pyspark.sql import SparkSession
from pyspark. sql.functions import *
import os

# Ruta base donde están los CSVs
base_path = "/Workspace/startup-death-oracle/data/bronze/"

# Listar archivos disponibles
files = os.listdir(base_path)
print("📁 Archivos en Bronze:")
for f in files:
    print(f"   ├── {f}")

📁 Archivos en Bronze:
   ├── Startup Failure (Health Care).csv
   ├── Startup Failure (Food and services).csv
   ├── Startup Failures.csv
   ├── Startup Failure (Finance and Insurance).csv
   ├── Startup Failure (Manufactures).csv
   ├── Startup Failure (Retail Trade).csv
   ├── Startup Failures (Information Sector).csv
   ├── startup data.csv


In [0]:
# Lista de archivos de failures (excluimos startup data.csv y el consolidado)
failure_files = [f for f in files if f.startswith("Startup Failure") and "Sector" not in f]
failure_files.append("Startup Failures (Information Sector).csv")  # Añadir el de Information

print(f"📊 Cargando {len(failure_files)} archivos de Startup Failures.. .\n")

# Cargar y unificar todos los CSVs
dfs = []
for file in failure_files:
    file_path = base_path + file
    df_temp = pd.read_csv(file_path)
    df_temp['source_file'] = file  # Añadir columna de origen
    dfs.append(df_temp)
    print(f"   ✅ {file}: {len(df_temp)} startups")

# Unificar en un solo DataFrame
df_failures = pd.concat(dfs, ignore_index=True)

print(f"\nTOTAL: {len(df_failures)} startups fallidas cargadas")
print(f"Columnas disponibles: {len(df_failures. columns)}")

📊 Cargando 7 archivos de Startup Failures.. .

   ✅ Startup Failure (Health Care).csv: 60 startups
   ✅ Startup Failure (Food and services).csv: 26 startups
   ✅ Startup Failures.csv: 815 startups
   ✅ Startup Failure (Finance and Insurance).csv: 47 startups
   ✅ Startup Failure (Manufactures).csv: 30 startups
   ✅ Startup Failure (Retail Trade).csv: 90 startups
   ✅ Startup Failures (Information Sector).csv: 156 startups

🎯 TOTAL: 1224 startups fallidas cargadas
📋 Columnas disponibles: 22


In [0]:
# Ver todas las columnas
print("COLUMNAS DISPONIBLES:\n")
for i, col in enumerate(df_failures. columns, 1):
    print(f"   {i:2}. {col}")

print("\n" + "="*50)
print("PRIMERAS 3 FILAS:")
print("="*50)
df_failures.head(3)

COLUMNAS DISPONIBLES:

    1. Name
    2. Sector
    3. Years of Operation
    4. What They Did
    5. How Much They Raised
    6. Why They Failed
    7. Takeaway
    8. Giants
    9. No Budget
   10. Competition
   11. Poor Market Fit
   12. Acquisition Stagnation
   13. Platform Dependency
   14. Monetization Failure
   15. Niche Limits
   16. Execution Flaws
   17. Trend Shifts
   18. Toxicity/Trust Issues
   19. Regulatory Pressure
   20. Overhype
   21. source_file
   22. High Operational Costs

PRIMERAS 3 FILAS:


,Name,Sector,Years of Operation,What They Did,How Much They Raised,Why They Failed,Takeaway,Giants,No Budget,Competition,Poor Market Fit,Acquisition Stagnation,Platform Dependency,Monetization Failure,Niche Limits,Execution Flaws,Trend Shifts,Toxicity/Trust Issues,Regulatory Pressure,Overhype,source_file,High Operational Costs
0,Aira Health,Health Care,2015-2019,Personalized asthma/allergy app,$12M,Small user base and cash shortage,Niche apps need big audiences,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,Startup Failure (Health Care).csv,NaN
1,Amino,Health Care,2013-2021,Doctor search and cost estimation,$45M,Lost to Zocdoc/GoodRx and slow adoption,Narrow focus beats broad,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Startup Failure (Health Care).csv,NaN
2,Arivale,Health Care,2015-2019,Personalized health coaching,$50M,High costs and low demand,Premium needs mass market,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,Startup Failure (Health Care).csv,NaN


In [0]:
# Seleccionar una startup de ejemplo
ejemplo = df_failures.iloc[0]

print("EJEMPLO DE STARTUP FALLIDA:")
print("=" * 50)
print(f"Nombre: {ejemplo['Name']}")
print(f"Sector: {ejemplo['Sector']}")
print(f"Anos operando: {ejemplo['Years of Operation']}")
print(f"Funding: {ejemplo['How Much They Raised']}")
print("-" * 50)
print(f"QUE HACIAN:\n{ejemplo['What They Did']}")
print("-" * 50)
print(f"POR QUE FALLARON:\n{ejemplo['Why They Failed']}")
print("-" * 50)
print(f"LECCION:\n{ejemplo['Takeaway']}")

EJEMPLO DE STARTUP FALLIDA:
Nombre: Aira Health
Sector: Health Care
Anos operando: 2015-2019
Funding: $12M
--------------------------------------------------
QUE HACIAN:
Personalized asthma/allergy app
--------------------------------------------------
POR QUE FALLARON:
Small user base and cash shortage
--------------------------------------------------
LECCION:
Niche apps need big audiences


In [0]:
print("ESTADISTICAS DEL DATASET")
print("=" * 50)

# Total de startups
print(f"Total startups fallidas: {len(df_failures)}")

# Por sector
print("\nSTARTUPS POR SECTOR:")
print("-" * 50)
sector_counts = df_failures['Sector']. value_counts()
for sector, count in sector_counts.items():
    print(f"   {sector}: {count}")

# Verificar valores nulos en campos clave
print("\nVALORES NULOS EN CAMPOS CLAVE:")
print("-" * 50)
campos_clave = ['Name', 'What They Did', 'Why They Failed', 'Takeaway']
for campo in campos_clave:
    nulos = df_failures[campo].isna().sum()
    print(f"   {campo}: {nulos} nulos")

ESTADISTICAS DEL DATASET
Total startups fallidas: 1224

STARTUPS POR SECTOR:
--------------------------------------------------
   Information: 526
   Retail Trade: 200
   Finance and Insurance: 97
   Health Care: 92
   Manufacturing: 76
   Accommodation and Food Services: 54
   Arts Entertainment and Recreation: 49
   Professional Scientific and Technical Services: 45
   Administrative and Support Services: 38
   Transportation and Warehousing: 26
   Real Estate and Rental and Leasing: 15
   Construction: 5
   Agriculture Forestry Fishing and Hunting: 1

VALORES NULOS EN CAMPOS CLAVE:
--------------------------------------------------
   Name: 0 nulos
   What They Did: 815 nulos
   Why They Failed: 815 nulos
   Takeaway: 815 nulos


Tenemos 815 registros con datos vacíos
Esto coincide con el archivo "Startup Failures.csv" (815 startups)
Probablemente tiene una estructura diferente. 

In [0]:
# Ver estructura del archivo Startup Failures. csv
df_main = pd.read_csv(base_path + "Startup Failures.csv")

print("ARCHIVO: Startup Failures. csv")
print("=" * 50)
print(f"Filas: {len(df_main)}")
print(f"\nCOLUMNAS:")
for i, col in enumerate(df_main.columns, 1):
    print(f"   {i}. {col}")

print("\n" + "=" * 50)
print("EJEMPLO DE ESTE ARCHIVO:")
print("=" * 50)
df_main.head(2)

ARCHIVO: Startup Failures. csv
Filas: 815

COLUMNAS:
   1. Name
   2. Sector
   3. Years of Operation

EJEMPLO DE ESTE ARCHIVO:


,Name,Sector,Years of Operation
0,99dresses,Retail Trade,3 (2010-2013)
1,Ahalife,Retail Trade,7 (2010-2017)


El archivo Startup Failures. csv solo tiene 3 columnas
Ya que es un resumen.
No tiene los campos de texto (What They Did, Why They Failed, Takeaway).
SOLUCION: Excluir este archivo
Vamos a trabajar solo con los archivos que tienen datos completos.


In [0]:
# Archivos con estructura completa (excluir Startup Failures.csv)
failure_files_completos = [
    "Startup Failure (Health Care).csv",
    "Startup Failure (Food and services).csv",
    "Startup Failure (Finance and Insurance).csv",
    "Startup Failure (Manufactures).csv",
    "Startup Failure (Retail Trade).csv",
    "Startup Failures (Information Sector).csv"
]

print(f"Cargando {len(failure_files_completos)} archivos con datos completos.. .\n")

# Cargar y unificar
dfs = []
for file in failure_files_completos:
    file_path = base_path + file
    df_temp = pd.read_csv(file_path)
    df_temp['source_file'] = file
    dfs. append(df_temp)
    print(f"   {file}: {len(df_temp)} startups")

# Unificar en un solo DataFrame
df_failures = pd.concat(dfs, ignore_index=True)

print(f"\nTOTAL: {len(df_failures)} startups con datos completos")

# Verificar que no hay nulos
nulos = df_failures['Why They Failed'].isna().sum()
print(f"Nulos en 'Why They Failed': {nulos}")

Cargando 6 archivos con datos completos.. .

   Startup Failure (Health Care).csv: 60 startups
   Startup Failure (Food and services).csv: 26 startups
   Startup Failure (Finance and Insurance).csv: 47 startups
   Startup Failure (Manufactures).csv: 30 startups
   Startup Failure (Retail Trade).csv: 90 startups
   Startup Failures (Information Sector).csv: 156 startups

TOTAL: 409 startups con datos completos
Nulos en 'Why They Failed': 0


Ahora cargar el dataset de EXITOS (startup data. csv)

In [0]:
df_success = pd. read_csv(base_path + "startup data.csv")

print("ARCHIVO: startup data.csv (STARTUPS EXITOSAS)")
print("=" * 50)
print(f"Filas: {len(df_success)}")
print(f"\nCOLUMNAS:")
for i, col in enumerate(df_success.columns, 1):
    print(f"   {i}. {col}")

print("\n" + "=" * 50)
print("EJEMPLO:")
print("=" * 50)
df_success.head(2)

ARCHIVO: startup data.csv (STARTUPS EXITOSAS)
Filas: 923

COLUMNAS:
   1. Unnamed: 0
   2. state_code
   3. latitude
   4. longitude
   5. zip_code
   6. id
   7. city
   8. Unnamed: 6
   9. name
   10. labels
   11. founded_at
   12. closed_at
   13. first_funding_at
   14. last_funding_at
   15. age_first_funding_year
   16. age_last_funding_year
   17. age_first_milestone_year
   18. age_last_milestone_year
   19. relationships
   20. funding_rounds
   21. funding_total_usd
   22. milestones
   23. state_code.1
   24. is_CA
   25. is_NY
   26. is_MA
   27. is_TX
   28. is_otherstate
   29. category_code
   30. is_software
   31. is_web
   32. is_mobile
   33. is_enterprise
   34. is_advertising
   35. is_gamesvideo
   36. is_ecommerce
   37. is_biotech
   38. is_consulting
   39. is_othercategory
   40. object_id
   41. has_VC
   42. has_angel
   43. has_roundA
   44. has_roundB
   45. has_roundC
   46. has_roundD
   47. avg_participants
   48. is_top500
   49. status

EJEMPLO:


,Unnamed: 0,state_code,latitude,longitude,zip_code,id,city,Unnamed: 6,name,labels,founded_at,closed_at,first_funding_at,last_funding_at,age_first_funding_year,age_last_funding_year,age_first_milestone_year,age_last_milestone_year,relationships,funding_rounds,funding_total_usd,milestones,state_code.1,is_CA,is_NY,is_MA,is_TX,is_otherstate,category_code,is_software,is_web,is_mobile,is_enterprise,is_advertising,is_gamesvideo,is_ecommerce,is_biotech,is_consulting,is_othercategory,object_id,has_VC,has_angel,has_roundA,has_roundB,has_roundC,has_roundD,avg_participants,is_top500,status
0,1005,CA,42.358880,-71.056820,92101,c:6669,San Diego,NaN,Bandsintown,1,1/1/2007,NaN,4/1/2009,1/1/2010,2.2493,3.0027,4.6685,6.7041,3,3,375000,3,CA,1,0,0,0,0,music,0,0,0,0,0,0,0,0,0,1,c:6669,0,1,0,0,0,0,1.00,0,acquired
1,204,CA,37.238916,-121.973718,95032,c:16283,Los Gatos,NaN,TriCipher,1,1/1/2000,NaN,2/14/2005,12/28/2009,5.1260,9.9973,7.0055,7.0055,9,4,40100000,1,CA,1,0,0,0,0,enterprise,0,0,0,1,0,0,0,0,0,0,c:16283,1,0,0,1,1,1,4.75,1,acquired


Ver la columna STATUS del dataset de startups

In [0]:
print("VALORES EN COLUMNA 'status':")
print("=" * 50)
print(df_success['status']. value_counts())

print("\n" + "=" * 50)
print("EJEMPLO DE STARTUP:")
print("=" * 50)
ejemplo = df_success.iloc[0]
print(f"Nombre: {ejemplo['name']}")
print(f"Ciudad: {ejemplo['city']}")
print(f"Categoria: {ejemplo['category_code']}")
print(f"Funding total: ${ejemplo['funding_total_usd']:,.0f}")
print(f"Rondas de funding: {ejemplo['funding_rounds']}")
print(f"Status: {ejemplo['status']}")

VALORES EN COLUMNA 'status':
status
acquired    597
closed      326
Name: count, dtype: int64

EJEMPLO DE STARTUP:
Nombre: Bandsintown
Ciudad: San Diego
Categoria: music
Funding total: $375,000
Rondas de funding: 3
Status: acquired


├── acquired (adquiridas): 597 startups  --> EXITO

├── closed (cerradas):     326 startups  --> FRACASO

RESUMEN DE LO QUE TENEMOS:

PARA EMBEDDINGS DE TEXTO (df_failures):

├── 409 startups con texto descriptivo

├── What They Did, Why They Failed, Takeaway

└── Perfecto para entrenar el "oraculo"

PARA ANALISIS NUMERICO (df_success):

├── 597 adquiridas (exito)

├── 326 cerradas (fracaso)  

├── Funding, milestones, rondas, etc.

└── Perfecto para modelo predictivo

Guardar df_failures limpio en SILVER

In [0]:
silver_path = "/Workspace/startup-death-oracle/data/silver/"

# Guardar failures con datos completos
df_failures.to_csv(silver_path + "failures_clean.csv", index=False)
print(f"Guardado: failures_clean.csv ({len(df_failures)} registros)")

# Guardar success data
df_success. to_csv(silver_path + "startups_data_clean.csv", index=False)
print(f"Guardado: startups_data_clean.csv ({len(df_success)} registros)")

print("\nArchivos guardados en Silver layer!")

Guardado: failures_clean.csv (409 registros)
Guardado: startups_data_clean.csv (923 registros)

Archivos guardados en Silver layer!


# 01_Bronze_Ingestion

## Datasets cargados

| Dataset | Registros | Descripcion |
|---------|-----------|-------------|
| failures_clean. csv | 409 | Startups fallidas con texto descriptivo |
| startups_data_clean.csv | 923 | Startups (597 adquiridas, 326 cerradas) |

## Columnas con flags (0/1)
Veremos muchas columnas con 0, 1. Son flags de carateristicas.

**Failures** - Causas de muerte: Giants, No Budget, Competition, Poor Market Fit, Platform Dependency, Monetization Failure, Execution Flaws, etc.

**Success** - Caracteristicas: is_CA, is_software, has_VC, has_roundA/B/C/D, is_top500, etc.

## Output
Archivos guardados en `/data/silver/`